In [1]:
!pip install pyautogen openai requests tavily-python crewai

In [2]:
import time
from tavily import TavilyClient
import google.generativeai as genai
import os
from pathlib import Path
import markdown
from crewai import Crew, Task, Agent

In [4]:
# Manually set API keys (Replace with your actual keys)
TAVILY_API_KEY = "tavily_api_key"
GEMINI_API_KEY = "gemini_api_key"

# Ensure API keys are available
if not TAVILY_API_KEY or not GEMINI_API_KEY:
    raise ValueError("Missing API keys. Set TAVILY_API_KEY and GEMINI_API_KEY manually.")

# Initialize Tavily client
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

# Initialize Gemini
genai.configure(api_key=GEMINI_API_KEY)

gemini_model = "gemini-1.5-pro"
gemini_generation_config = {
    "temperature": 0.7,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
}

# Function to perform Tavily search
def tavily_search(query):
    try:
        results = tavily_client.search(query)
        return results
    except Exception as e:
        print(f"Error during Tavily search: {e}")
        return {"error": str(e), "results": []}

# Function to generate content with Gemini
def generate_with_gemini(prompt, system_message=""):
    full_prompt = f"{system_message}\n\n{prompt}" if system_message else prompt
    model = genai.GenerativeModel(model_name=gemini_model, generation_config=gemini_generation_config)
    try:
        response = model.generate_content(full_prompt)
        return response.text
    except Exception as e:
        print(f"Error generating content: {e}")
        return "Request failed. Please check API quota or connection."

# Function to display markdown content
def display_markdown(content):
    print("\n" + markdown.markdown(content) + "\n")

# Define Agents
research_agent = Agent(
    name="Researcher",
    role="Research Specialist",
    goal="Gather latest and relevant information on a given topic.",
    backstory="An AI-powered researcher capable of retrieving up-to-date information."
)

writer_agent = Agent(
    name="Writer",
    role="Article Writer",
    goal="Generate well-structured articles from the research findings.",
    backstory="An AI writer skilled in creating engaging and informative content."
)

translator_agent = Agent(
    name="Translator",
    role="Language Translator",
    goal="Translate articles while preserving meaning and tone.",
    backstory="A language expert proficient in high-quality translations."
)

# Define Tasks
research_task = Task(
    description="Perform research on the given topic using real-time search APIs.",
    agent=research_agent,
    expected_output="A summary of the latest information gathered from online sources."
)

write_task = Task(
    description="Write a structured article based on research findings.",
    agent=writer_agent,
    depends_on=[research_task],
    expected_output="A well-structured article based on the research."
)

translate_task = Task(
    description="Translate the article into the target language.",
    agent=translator_agent,
    depends_on=[write_task],
    expected_output="A translated version of the article maintaining accuracy and tone."
)

# Main execution
def main():
    print("\n=== Research and Translation Tool ===\n")
    topic = input("Enter the topic for research: ")
    target_language = input("Enter the target language for translation: ")
    writing_style = input("Enter the desired writing style (optional, press enter to skip): ") or "formal"

    print(f"\nResearching '{topic}'...")
    research_results = tavily_search(topic)
    if "error" in research_results and not research_results.get("results"):
        print(f"Research failed: {research_results.get('error')}")
        return
    print(f"Research complete. Found {len(research_results.get('results', []))} results.")

    print(f"\nGenerating article in {writing_style} style...")
    article_prompt = f"Write a comprehensive article about '{topic}' based on these research findings:\n\n{research_results}"
    article = generate_with_gemini(article_prompt)

    if article.startswith("Request failed"):
        print("Failed to generate article.")
        return

    print("\n=== Generated Article ===\n")
    display_markdown(article)

    article_filename = "original_article.md"
    with open(article_filename, "w", encoding="utf-8") as f:
        f.write(article)
    print(f"\nOriginal article saved to {article_filename}")

    print(f"\nTranslating article to {target_language}...")
    translation_prompt = f"Translate this article into {target_language}. Maintain the formal structure and professional tone:\n\n{article}"
    translated_article = generate_with_gemini(translation_prompt)

    if translated_article.startswith("Request failed"):
        print("Failed to translate article.")
        return

    print("\n=== Translated Article ===\n")
    display_markdown(translated_article)

    translated_filename = f"translated_article_{target_language}.md"
    with open(translated_filename, "w", encoding="utf-8") as f:
        f.write(translated_article)
    print(f"\nTranslated article saved to {translated_filename}")
    print("\nProcess completed successfully!")

# Run the main function
if __name__ == "__main__":
    main()




=== Research and Translation Tool ===

Enter the topic for research: gen ai
Enter the target language for translation: hindi
Enter the desired writing style (optional, press enter to skip): 

Researching 'gen ai'...
Research complete. Found 5 results.

Generating article in formal style...

=== Generated Article ===


<h2>The Rise of Gen AI: Creating, Innovating, and Transforming</h2>
<p>Generative AI (GenAI or GAI) is rapidly changing the landscape of artificial intelligence and its applications across various industries.  No longer limited to science fiction, GenAI empowers machines to create new content, ranging from text and images to music and even code, by learning patterns from existing data.  This article explores the core concepts, capabilities, and impact of this transformative technology.</p>
<p><strong>What is Generative AI?</strong></p>
<p>At its core, GenAI is a subset of artificial intelligence that leverages generative models to produce novel data.  Unlike traditional 